In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import statsmodels.formula.api as smf
import scipy.stats as stats 
import seaborn as sns

print ('reading data file...')
data = pd.read_csv('nesarc_pds.csv', low_memory=False)
data.columns = map(str.upper, data.columns)

# bug fix for display formats to avoid run time errors - put after code for loading data above
pd.set_option('display.float_format', lambda x:'%f'%x)
pd.set_option('display.max_rows', None)
# Current drinkers(CONSUMER  -  DRINKING STATUS ) Either 1 (yes) or 2(no) to (S7Q31A -  EVER DRANK ALCOHOL TO AVOID SOCIAL PHOBIA)
drinkerstemp=data[(data['CONSUMER'] ==1) & ((data['S7Q31A']=='1') | (data['S7Q31A']=='2'))]

#Get rid of everything unneeded 
drinkers = drinkerstemp[['SEX','S7Q31A','S2AQ8B','S2AQ8C','S2AQ10','S2BQ1A2','S2BQ1A4','S2BQ1A7', 'S2BQ1A8','S2BQ3B']].copy()

del drinkerstemp 
del data

for col in drinkers: # Convert columns to numeric and replace 99's and nulls
    drinkers[col] = drinkers[col].convert_objects(convert_numeric=True)
    drinkers[col]=drinkers[col].replace(99 ,np.nan).fillna(np.nan)

for col in ['S2BQ1A2','S2BQ1A4','S2BQ1A7']: # Set missing values to Nan
    drinkers[col]=drinkers[col].replace(9 ,np.nan).fillna(np.nan)
    
drinkers['S7Q31A'] = drinkers['S7Q31A'].map({1:1,2:0})
#Assign 
drinkers['DRUNK_CNT'] = drinkers['S2AQ10'].map({1:10,2:9,3:8,4:7,5:6,6:5,7:4,8:3,9:2,10:1})
    
#Testing a linear regression model
    
print(drinkers['S7Q31A'].value_counts())
    
lin_reg = smf.ols('S2AQ8B ~ S7Q31A',data=drinkers).fit()
print(lin_reg.summary())   

In [2]:
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 S2AQ8B   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     115.0
Date:                Sat, 30 Apr 2016   Prob (F-statistic):           1.98e-26
Time:                        21:49:07   Log-Likelihood:                -8702.8
No. Observations:                3491   AIC:                         1.741e+04
Df Residuals:                    3489   BIC:                         1.742e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.3786      0.053     44.675      0.000         2.274     2.483
S7Q31A         1.5614      0.146     10.726      0.000         1.276     1.847
==============================================================================
Omnibus:                     6204.398   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15197795.067
Skew:                          12.232   Prob(JB):                         0.00
Kurtosis:                     325.310   Cond. No.                         3.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
model2 = 